In [1]:
## gpt 4o ehr review -- discharge summary 

In [2]:
import pandas as pd
df = pd.read_csv('results.csv')

In [3]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'patientdurablekey',
       'encounterkey', 'ArrivalDateKey', 'DepartureDateKeyValue',
       'DepartureDateKey', 'DispositionDateKeyValue',
       'primarychiefcomplaintname', 'primaryeddiagnosisname', 'sex',
       'birthdate', 'firstrace', 'preferredlanguage',
       'highestlevelofeducation', 'maritalstatus', 'Age',
       'Discharge_Summary_Date', 'Discharge_Summary_Note_Key',
       'Progress_Note_Date', 'Progress_Note_Key', 'HP_Note_Date',
       'HP_Note_Key', 'Echo_Date', 'Echo_Key', 'Imaging_Date', 'Imaging_Key',
       'Consult_Date', 'Consult_Key', 'ED_Provider_Notes_Date',
       'ED_Provider_Notes_Key', 'ECG_Date', 'ECG_Key',
       'Discharge_Summary_Text', 'Progress_Note_Text', 'HP_Note_Text',
       'Echo_Text', 'Imaging_Text', 'Consult_Text', 'ECG_Text',
       'ED_Provider_Notes_Text', 'One_Sentence_Extracted', 'note_count',
       'acuitylevel', 'eddisposition'],
      dtype='object')

In [4]:
df.shape

(3984, 46)

In [9]:
import os
import re
import json
import base64
import requests
import time
import urllib.parse


API_KEY = 'x'  ##### Paste your API key between the quotes #####
API_VERSION = '2024-06-01'  # For the most recent production release: https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release
RESOURCE_ENDPOINT = 'https://unified-api.ucsf.edu/general'  # no trailing slash--this is used by libraries as a partial URL
DEPLOYMENT_NAME = "gpt-4o-2024-08-06"   # no trailing slash--this is used by libraries as a partial URL

In [14]:
import pandas as pd
import requests
import time
import os
import json
from tqdm import tqdm  # For progress bar

# Function to get a summary from Azure OpenAI
def get_summary(chief_complaint, discharge_summary, age, sex):
    # Check if key fields are missing
    if pd.isna(chief_complaint) or pd.isna(discharge_summary):
        return None  # Skip if any key field is missing
    
    url = f"{RESOURCE_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version={API_VERSION}"
    
    headers = {
        "Content-Type": "application/json",
        "api-key": API_KEY
    }
    
    payload = {
        "messages": [
            {"role": "system", "content": "You are an experienced emergency department (ED) physician creating a one-liner for a NEW patient who has just arrived at the ED. The patient's past medical records are available to you. Your task is to summarize the patient's relevant PAST medical history and end with their CURRENT chief complaint that is given with no adjectives about the chief complaint as you can NOT assume anything about their current condition. All notes and medical records provided are from PAST encounters, not the current visit."},
            {"role": "user", "content": f"Create a concise one-liner summary for a patient who has just arrived at the Emergency Department. The one-liner must:\n\n"
                                      f"1. Start with demographic information (age, sex). Example of a one liner:  80 y.o. old male, with h/o of HFpEF (EF 55-60% 05/20/22), HTN, HLD, and bipolar disorder presenting with shortness of breath. \n"
                                      f"2. Include a concise summary of relevant PAST medical history from previous visits/notes\n"
                                      f"3. End with just CURRENT presenting chief complaint that is not capitilized in the summary and does have additional information regarding the chief complaint: '{chief_complaint}'\n\n"
                                      f"IMPORTANT: Everything in the notes is from PAST encounters. The patient is NOW presenting with a NEW complaint: '{chief_complaint}'.\n\n"
                                      f"Age: {age}\n"
                                      f"Sex: {sex}\n"
                                      f"PAST Medical Records:\n{discharge_summary}"}
        ],
        "temperature": 0.1,
        "max_tokens": 4096
    }
    
    retries = 0
    while retries < 3:
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()  # Raise an error for non-200 responses
            return response.json()["choices"][0]["message"]["content"].strip()
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}. Retrying {retries+1}/3...")
            time.sleep(5)  # Wait before retrying
            retries += 1
    
    return None  # Return None if all retries fail

# Function to process dataframe with checkpoint saving
def process_dataframe_with_checkpoints(df, checkpoint_file="new_processing_checkpoint.json", output_file="new_4o_ehr_review.csv", batch_size=10):
    # Check if there's a checkpoint to resume from
    start_index = 0
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint_data = json.load(f)
            start_index = checkpoint_data.get('last_processed_index', 0) + 1
            print(f"Resuming from index {start_index}")
            
            # If there's a partially processed CSV, load it
            if os.path.exists(output_file):
                saved_df = pd.read_csv(output_file)
                # Ensure it has the Generated_Summary column
                if 'Generated_Summary' not in saved_df.columns:
                    saved_df['Generated_Summary'] = None
                # Transfer any already processed summaries
                for idx in range(start_index):
                    if idx < len(df) and idx < len(saved_df):
                        if not pd.isna(saved_df.loc[idx, 'Generated_Summary']):
                            df.loc[idx, 'Generated_Summary'] = saved_df.loc[idx, 'Generated_Summary']

    # Initialize Generated_Summary column if it doesn't exist
    if 'Generated_Summary' not in df.columns:
        df['Generated_Summary'] = None
    
    # Process in batches with progress bar
    total_rows = len(df)
    progress_bar = tqdm(total=total_rows, initial=start_index, desc="Processing records")
    
    for i in range(start_index, total_rows):
        row = df.iloc[i]
        
        # Process the current row
        summary = get_summary(
            row["primarychiefcomplaintname"], 
            row["Discharge_Summary_Text"],
            row["Age"],
            row["sex"]
        )
        
        # Update dataframe
        df.loc[i, 'Generated_Summary'] = summary
        
        # Update progress bar
        progress_bar.update(1)
        
        # Save checkpoint and intermediate results after each batch
        if (i + 1) % batch_size == 0 or i == total_rows - 1:
            # Save checkpoint
            with open(checkpoint_file, 'w') as f:
                json.dump({'last_processed_index': i}, f)
            
            # Save current results
            df.to_csv(output_file, index=False)
            print(f"\nCheckpoint saved at index {i}")
    
    progress_bar.close()
    print(f"Processing complete. Results saved to {output_file}")
    
    # Clean up checkpoint file when done
    if os.path.exists(checkpoint_file):
        os.remove(checkpoint_file)
    
    return df

# Apply the processing function to the dataframe
df = process_dataframe_with_checkpoints(df)

Processing records:   0%|          | 10/3984 [00:14<1:36:23,  1.46s/it]


Checkpoint saved at index 9


Processing records:   1%|          | 20/3984 [00:34<1:31:38,  1.39s/it]


Checkpoint saved at index 19


Processing records:   1%|          | 30/3984 [00:52<1:27:40,  1.33s/it]

...


Checkpoint saved at index 3969


Processing records: 100%|█████████▉| 3980/3984 [2:07:32<00:04,  1.20s/it]


Checkpoint saved at index 3979


Processing records: 100%|██████████| 3984/3984 [2:07:48<00:00,  1.92s/it]


Checkpoint saved at index 3983
Processing complete. Results saved to new_4o_ehr_review.csv


In [5]:
check = pd.read_csv('new_4o_ehr_review.csv')

In [6]:
check.shape

(3984, 47)